# Kaggle Ship Create Submission File on Kaggle Environment

Link to competition: https://www.kaggle.com/c/airbus-ship-detection

This notebook was converted from my prior Kaggle notebook.  Migrated to TF 2.x and converted various methods to be more native TF.  This will create a Kaggle submission file.  It uses a trained model and the normal tensor data set processing.  In the bottom cells you can check processing.  The model I created in the "Ship_Create_Model_V1" uses image dims of (224,224,3), so this notebook assums the same.  You can alter the expected dims in the configuration settings.  The final mask/rle encoded result is resized from (224, 224) to (768, 768).  This can also be changed.

In [0]:
# Change to True if using Kaggle environment....
USING_KAGGLE = True

In [0]:
# Normal includes...
from __future__ import absolute_import, division, print_function, unicode_literals

import os, sys, random, warnings, time, copy, csv
import numpy as np 
from tqdm import notebook, trange, tqdm

import IPython.display as display
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import tensorflow as tf
print(tf.__version__)

from tensorflow.keras.models import load_model 
from skimage.morphology import binary_opening, disk, label

# This allows the runtime to decide how best to optimize CPU/GPU usage
AUTOTUNE = tf.data.experimental.AUTOTUNE

In [0]:
# Config class that wraps global variable access, using personal libs is a pain in Kaggle, so copied in the class
# Not all of the global vars are used, easier to jsut copy class over from lib

class GlobalParms(object):

    def __init__(self, **kwargs):
        self.keys_and_defaults = {
         "MODEL_NAME": "",  # if you leave .h5 off, puts into a subdirectory
         "ROOT_PATH": "",  # Location of the data for storing any data or files
         "TRAIN_DIR": "",  # Subdirectory in the Root for Training files
         "TEST_DIR": "",  # Optional subdirectory in  Root for Testing file
         "SUBMISSION_PATH": None,  # Optional subdirectory for Contest files
         "MODEL_PATH": None,  # Optional, subdirectory for saving/loading model
         "TRAIN_PATH": None,  # Subdirectory in the Root for Training files
         "TEST_PATH": None,  # Optional subdirectory in  Root for Testing file
         "SMALL_RUN": False,   # Optional, run size will be reduced
         "NUM_CLASSES": 0,  # Number of classes
         "CLASS_NAMES": [],  # list of class names
         "IMAGE_ROWS": 0,  # Row size of the image
         "IMAGE_COLS": 0,  # Col size of the image
         "IMAGE_CHANNELS": 0,  # Num of Channels, 1 for Greyscale, 3 for color
         "BATCH_SIZE": 0,  # Number of images in each batch
         "EPOCS": 0,  # Max number of training EPOCS
         "ROW_SCALE_FACTOR": 1,  # Optional, allows scaling of an image.
         "COL_SCALE_FACTOR": 1,  # Optional, allows scaling of an image.
         "IMAGE_EXT": ".jpg",  # Extent of the image file_ext
         # Optional, default is np.float64, reduce memory by using np.float32
         # or np.float16
         "IMAGE_DTYPE": np.float32,
         # Optional, change default if needed, can save memory space
         "Y_DTYPE": np.int,
         "LOAD_MODEL": False,  # Optional, If you want to load a saved model
         "SUBMISSION": "submission.csv",  # Optional, Mainly used for Kaggle
         "METRICS": ['accuracy'],  # ['categorical_accuracy'], ['accuracy']
         "FINAL_ACTIVATION": 'sigmoid',  # sigmoid, softmax
         "LOSS": ""  # 'binary_crossentropy', 'categorical_crossentropy'
        }

        self.__dict__.update(self.keys_and_defaults)
        self.__dict__.update((k, v) for k, v in kwargs.items()
                             if k in self.keys_and_defaults)

        # Automatically reduce the training parms, change as needed
        if self.__dict__["SMALL_RUN"]:
            self.__dict__["BATCH_SIZE"] = 1
            self.__dict__["EPOCS"] = 2
            self.__dict__["ROW_SCALE_FACTOR"] = 1
            self.__dict__["COL_SCALE_FACTOR"] = 1

        # Use configuration items to create real ones
        self.__dict__["SCALED_ROW_DIM"] = \
            np.int(self.__dict__["IMAGE_ROWS"] /
                   self.__dict__["ROW_SCALE_FACTOR"])

        self.__dict__["SCALED_COL_DIM"] =  \
            np.int(self.__dict__["IMAGE_COLS"] /
                   self.__dict__["COL_SCALE_FACTOR"])

        if self.__dict__["TRAIN_PATH"] is None:  # Not passed, so set it
            self.__dict__["TRAIN_PATH"] = \
                os.path.join(self.__dict__["ROOT_PATH"],
                             self.__dict__["TRAIN_DIR"])

        if self.__dict__["TEST_PATH"] is None:  # Not passed, so set it
            self.__dict__["TEST_PATH"] = \
                os.path.join(self.__dict__["ROOT_PATH"],
                             self.__dict__["TEST_DIR"])

        if self.__dict__["SUBMISSION_PATH"] is None:  # Not passed, so set
            self.__dict__["SUBMISSION_PATH"] = \
                os.path.join(self.__dict__["ROOT_PATH"],
                             self.__dict__["SUBMISSION"])
        else:
            self.__dict__["SUBMISSION_PATH"] = \
                os.path.join(self.__dict__["SUBMISSION_PATH"],
                             self.__dict__["SUBMISSION"])

        if self.__dict__["MODEL_PATH"] is None:  # Not passed, so set it
            self.__dict__["MODEL_PATH"] = \
                os.path.join(self.__dict__["ROOT_PATH"],
                             self.__dict__["MODEL_NAME"])
        else:
            self.__dict__["MODEL_PATH"] = \
                os.path.join(self.__dict__["MODEL_PATH"],
                             self.__dict__["MODEL_NAME"])

        self.__dict__["IMAGE_DIM"] = \
            (self.__dict__["SCALED_ROW_DIM"],
             self.__dict__["SCALED_COL_DIM"],
             self.__dict__["IMAGE_CHANNELS"])

        if self.__dict__["IMAGE_CHANNELS"] == 1:
            self.__dict__["COLOR_MODE"] = "grayscale"
        else:
            self.__dict__["COLOR_MODE"] = "rgb"

    def set_train_path(self, train_path):
        self.__dict__["TRAIN_PATH"] = train_path

    def set_class_names(self, class_name_list):
        self.__dict__["CLASS_NAMES"] = class_name_list

        if self.__dict__["NUM_CLASSES"] != \
           len(self.__dict__["CLASS_NAMES"]):
            raise ValueError("ERROR number of classses do not match, Classes: "
                             + str(self.__dict__["NUM_CLASSES"])
                             + " Class List: "
                             + str(self.__dict__["CLASS_NAMES"]))

    def print_contents(self):
        print(self.__dict__)

    def print_key_value(self):
        for key, value in self.__dict__.items():
            print(key, ":", value)



## Various Methods

In [ ]:
# Set these to match your environment

if USING_KAGGLE:
    ROOT_PATH = "../input/airbus-ship-detection/"  ###### CHANGE FOR SPECIFIC ENVIRONMENT
else:
    ROOT_PATH = "/Users/john/Documents/ImageData/KaggleShip/"  ###### CHANGE FOR SPECIFIC ENVIRONMENT

# Establish global dictionary
parms = GlobalParms(ROOT_PATH=ROOT_PATH,
                    MODEL_NAME="airModel.h5",
                    MODEL_PATH="",
                    TEST_DIR="test_v2", 
                    IMAGE_ROWS=224,
                    IMAGE_COLS=224,
                    IMAGE_CHANNELS=3,
                    SUBMISSION="submission.csv",
                    IMAGE_EXT=".jpg")

parms.print_contents()

In [0]:

def multi_rle_encode(img, **kwargs):
    '''
    Encode connected regions as separated masks
    '''
    labels, num_labels = label(img, return_num=True)
    #print("Labels ", num_labels, " image dim: ", img.ndim)
    #for i in range(num_labels):
    #    print("label ", i, " nonzero ", np.sum(labels[i]))
        
    if img.ndim > 2:
        return [rle_encode(np.sum(labels==k, axis=2), **kwargs) for k in np.unique(labels[labels>0])]
    else:
        return [rle_encode(labels==k, **kwargs) for k in np.unique(labels[labels>0])]
    
# ref: https://www.kaggle.com/paulorzp/run-length-encode-and-decode
def rle_encode(img, min_max_threshold=1e-3, max_mean_threshold=None):
    '''
    img: numpy array, 1 - mask, 0 - background
    Returns run length as string formated
    
    A return of None means there is nothing to create, no masks.  No masks are added back at the very end.
    This means that only masks are returned at first - helps with understanding how accurate.
    '''
    #print("rle_encode ", np.count_nonzero(img))
    if np.count_nonzero(img) < 25:
        return None
    if np.max(img) < min_max_threshold:
        return None ## no need to encode if it's all zeros
    if max_mean_threshold and np.mean(img) > max_mean_threshold:
        return None ## ignore overfilled mask
    
    pixels = img.T.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

def rle_decode(mask_rle, shape=(768, 768)):
    '''
    mask_rle: run-length as string formated (start length)
    shape: (height,width) of array to return 
    Returns numpy array, 1 - mask, 0 - background
    '''
    
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(shape).T  # Needed to align to RLE direction

def masks_as_image(in_mask_list, shape=(768, 768)):
    """Take the individual ship masks and create a single mask array for all ships"""
    
    all_masks = np.zeros(shape, dtype = np.int16)
    #print("All masks ", all_masks.shape)
    #if isinstance(in_mask_list, list):
    for mask in in_mask_list:
        if isinstance(mask, str):
            #print('calling decode', mask)
            all_masks += rle_decode(mask, shape)
    return np.expand_dims(all_masks, -1)


In [0]:
# Shows image and masks
def show_batch_mask(display_list):
    plt.figure(figsize=(15, 15))

    title = ['Input Image', 'True Mask', 'Predicted Mask']

    for i in range(len(display_list)):
        #print(display_list[i].shape)
        plt.subplot(1, len(display_list), i+1)
        plt.title(title[i])
        plt.imshow(tf.keras.preprocessing.image.array_to_img(display_list[i]))
        plt.axis('off')
    plt.show()
    
# Shows the images and executes the model for predictions
def show_test_predictions(model, dataset=None,  num=1):
    if dataset:
        for image, image_id in dataset.take(num):
            #print(image.shape)
            pred_mask = model.predict(image)[0]
            pred_mask = np.where(pred_mask > 0.5, 1, 0)
            show_batch_mask([image[0], pred_mask])
    else:
        show_batch_mask([sample_image,
             create_mask(model.predict(sample_image[tf.newaxis, ...]))])

In [0]:
# Used for TensorFlow Datasets

# Decode the image, convert to float, normalize by 255 and resize
def decode_img(image: tf.Tensor) -> tf.Tensor:
    # convert the compressed string to a 3D uint8 tensor
    image = tf.image.decode_jpeg(image, channels=parms.IMAGE_CHANNELS)
    # Use `convert_image_dtype` to convert to floats in the [0,1] range.
    image = tf.image.convert_image_dtype(image, parms.IMAGE_DTYPE)
    return image

# used by dataset to load images
def process_test_image_id(image_id: tf.Tensor) -> tf.Tensor:
    file_path = parms.TEST_PATH + "/" + image_id
    # load the raw data from the file as a string
    image = tf.io.read_file(file_path)
    image = decode_img(image)
    image = tf.image.resize(image, [parms.IMAGE_ROWS, parms.IMAGE_COLS])
    return image, image_id


## Build Tensor Dataset

You can change the TEST_DIR in the config to use the training set if you want to see actual masks.

In [0]:
test_files = np.array(os.listdir(parms.TEST_PATH))
print("Test: ", len(test_files), " ", test_files[0])

# modify to reduce the number of images processed, or comment out for full list
test_files = test_files[:100]

In [0]:
# Create Dataset from pd
test_dataset = tf.data.Dataset.from_tensor_slices(test_files)

# Verify image paths were loaded, all should be ok, but I've found a double check saves time later....
for image_id in test_dataset.take(2):
    print(image_id.numpy().decode("utf-8"))

# map training images to processing
test_dataset = test_dataset.map(process_test_image_id, num_parallel_calls=AUTOTUNE)

# Verify the mapping worked.  Both the actual image and the image_id should be returned
for image, image_id in test_dataset.take(1):
    print("Image ID: ", image_id.numpy().decode("utf-8"))
    print("Image shape: {}  Max: {}  Min: {}".format(image.numpy().shape, np.max(image.numpy()), np.min(image.numpy())))
    some_image = image.numpy()

#show_batch_mask([some_image, some_mask])

test_dataset = test_dataset.batch(1).repeat()

In [0]:
# Show the images, final check before running predictions.  Can change the take() number and execute multiple times
for image, image_id in test_dataset.take(1):
    sample_image= image[0]
show_batch_mask([sample_image])

In [0]:
# If a custom loss was used in training, need to have it here for model loading.  I included the ones I have been 
# playing with...

# https://lars76.github.io/neural-networks/object-detection/losses-for-segmentation/
def dice_loss(y_true, y_pred):
    numerator = 2 * tf.reduce_sum(y_true * y_pred, axis=(1,2,3))
    denominator = tf.reduce_sum(y_true + y_pred, axis=(1,2,3))
    return tf.reshape(1 - numerator / denominator, (-1, 1, 1))

def combo_loss(y_true, y_pred):
    def dice_loss(y_true, y_pred):
        numerator = 2 * tf.reduce_sum(y_true * y_pred, axis=(1,2,3))
        denominator = tf.reduce_sum(y_true + y_pred, axis=(1,2,3))
        return tf.reshape(1 - numerator / denominator, (-1, 1, 1))
    return tf.keras.losses.binary_crossentropy(y_true, y_pred, from_logits=True) + dice_loss(y_true, y_pred)

In [0]:
# CHANGE THIS TO WHERE YOUR MODEL IS LOCATED...

model_path = parms.MODEL_PATH
seg_model = load_model(model_path, custom_objects={'combo_loss': combo_loss}) # Load the trained model

In [0]:
show_test_predictions(seg_model, test_dataset, 15)

In [0]:
# Build predictions.  CV2 is used to resize the predicted mask.
import cv2

# predict and encode the result.  If None, then ignore the image, no ships were found or ships were too small.
def pred_encode(image, image_id, model, **kwargs):
    cur_rles = []
    pred_mask = model.predict(image)[0]
    
    # need to enlarge the mask to 768,768 for final prediction
    # for testing, you can comment this out and leave at 224,224
    pred_mask = cv2.resize(pred_mask, dsize=(768,768), interpolation=cv2.INTER_CUBIC)
    
    pred_mask = np.where(pred_mask > 0.5, 1, 0) 
    #print(image_id, np.count_nonzero(pred_mask))
    cur_rles += multi_rle_encode(pred_mask, **kwargs)        
    return [[image_id, rle] for rle in cur_rles if rle is not None]

# Loop through the dataset and process the images
def build_test_masks(dataset, model, steps, test):
    if test == True:
        steps = 2
        
    test_pred = []
    for image_tf, image_id_tf in tqdm(dataset.take(steps)):
        image = image_tf.numpy()
        image_id = image_id_tf.numpy()[0].decode("utf-8")
        if test:
            print("Image ID: ", image_id)
            print("Image shape: {}  Max: {}  Min: {}".format(image.shape, np.max(image), np.min(image)))

        test_pred += pred_encode(image, image_id, model, min_max_threshold=1.0)
            
    return test_pred

In [0]:
# Build the list of results, only images that had ships were returned

steps = len(test_files)
test_pred = build_test_masks(test_dataset, seg_model, steps=steps, test=False)

# Submission Notes...

There is additional post processing to clean-up the mask that I did not show.  Basically, there is a balance between removing random predictions vs leaving them.  The small dots or "ships" could be actual ships or they could be bad predictions.  This only works on grayscale, try something like a dialation followed by a erosion.  (This is a closing operation.)

https://scikit-image.org/docs/dev/api/skimage.morphology.html

from skimage.morphology import erosion, dilation, disk
- selem = disk(6)
- return erosion(dilation(mask, selem), selem)

In [0]:
# Create dataframe from result list
sub = pd.DataFrame(test_pred)
sub.columns = ['ImageId', 'EncodedPixels']
sub = sub[sub.EncodedPixels.notnull()]
sub

In [0]:
# Raw final display of images and predictions from submission file
# Onle last check....

from skimage.io import imread

def masks_using_color(in_mask_list, shape=(768,768)):
    # Take the individual ship masks and create a color mask array for each ships
    all_masks = np.zeros(shape, dtype = np.float)
    scale = lambda x: (len(in_mask_list)+x+1) / (len(in_mask_list)*2) ## scale the heatmap image to shift 
    for i,mask in enumerate(in_mask_list):
        if isinstance(mask, str):
            all_masks[:,:] += scale(i) * rle_decode(mask, shape=shape)
    return all_masks

TOP_PREDICTIONS=5
fig, m_axs = plt.subplots(TOP_PREDICTIONS, 2, figsize = (9, TOP_PREDICTIONS*5))
[c_ax.axis('off') for c_ax in m_axs.flatten()]

for (ax1, ax2), image_name in zip(m_axs, sub.ImageId.unique()[:TOP_PREDICTIONS]):
    image = imread(os.path.join(parms.TEST_PATH, image_name))
    image = np.expand_dims(image, 0)/255.0
    ax1.imshow(image[0])
    ax1.set_title('Image: ' + image_name)
    ax2.imshow(masks_using_color(sub.query('ImageId=="{}"'.format(image_name))['EncodedPixels']))
    ax2.set_title('Prediction')

In [0]:
# Create final csv file
# This is where images that did not have any predicted ships are added - join with sample_submission file
sub1 = pd.read_csv(parms.ROOT_PATH + 'sample_submission_v2.csv')
sub1 = pd.DataFrame(np.setdiff1d(sub1['ImageId'].unique(), sub['ImageId'].unique(), assume_unique=True), columns=['ImageId'])
sub1['EncodedPixels'] = None
print(len(sub1), len(sub))

sub = pd.concat([sub, sub1])
print(len(sub), "  Path: ", parms.SUBMISSION_PATH)
sub.to_csv(parms.SUBMISSION_PATH, index=False)
sub.head()